<a href="https://colab.research.google.com/github/yastiaisyah/DataSynthesis/blob/main/denoising_vae_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.datasets import MNIST
from torchvision import transforms
import torch.nn.init as init

# Define data transforms
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load MNIST dataset
mnist = MNIST(root='./data', train=True, transform=transform, download=True)

# Normalize the data to [0, 1] range
mnist.data = mnist.data.float() / 255.0

# Parameters
mb_size = 32
z_dim = 5
X_dim = mnist.data.size(1) * mnist.data.size(2)  # Flattened image dimensions
h_dim = 128
lr = 1e-3
noise_factor = .25

# Define Q, P, sample_z, and other functions here

def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / np.sqrt(in_dim / 2.)
    return Variable(torch.randn(*size) * xavier_stddev, requires_grad=True)

# Define Q(z|X)
Whz_mu = xavier_init(size=[h_dim, z_dim])
bhz_mu = Variable(torch.zeros(z_dim), requires_grad=True)

Whz_var = xavier_init(size=[h_dim, z_dim])
bhz_var = Variable(torch.zeros(z_dim), requires_grad=True)

def sample_z(mu, log_var):
    eps = Variable(torch.randn(mu.size()))
    return mu + torch.exp(log_var / 2) * eps

def Q(X):
    h = nn.ReLU()(X @ Wxh + bxh.repeat(X.size(0), 1))
    z_mu = h @ Whz_mu + bhz_mu.repeat(h.size(0), 1)
    z_var = h @ Whz_var + bhz_var.repeat(h.size(0), 1)
    return z_mu, z_var

# Define P(X|z)
Wzh = xavier_init(size=[z_dim, h_dim])
bzh = Variable(torch.zeros(h_dim), requires_grad=True)

Whx = xavier_init(size=[h_dim, X_dim])
bhx = Variable(torch.zeros(X_dim), requires_grad=True)

Wxh = xavier_init(size=[X_dim, h_dim])
bxh = Variable(torch.zeros(h_dim), requires_grad=True)

def P(z):
    h = nn.ReLU()(z @ Wzh + bzh.repeat(z.size(0), 1))
    X = nn.Sigmoid()(h @ Whx + bhx.repeat(h.size(0), 1))
    return X


# Training
params = [Wxh, bxh, Whz_mu, bhz_mu, Whz_var, bhz_var,
          Wzh, bzh, Whx, bhx]

solver = optim.Adam(params, lr=lr)
c = 0  # Counter for saving images

"""1000000"""
for it in range(100000):
    indices = torch.LongTensor(np.random.choice(mnist.data.size(0), mb_size))
    X = mnist.data[indices].view(mb_size, -1)  # Flatten the input image

    X = Variable(X)

    # Add noise
    X_noise = X + noise_factor * Variable(torch.randn(X.size()))
    X_noise.data.clamp_(0., 1.)

    # Forward
    z_mu, z_var = Q(X_noise)
    z = sample_z(z_mu, z_var)
    X_sample = P(z)

    # Loss calculation
    recon_loss = F.binary_cross_entropy(X_sample, X, size_average=False) / mb_size
    kl_loss = torch.mean(0.5 * torch.sum(torch.exp(z_var) + z_mu**2 - 1. - z_var, 1))
    loss = recon_loss + kl_loss

    # Backward
    solver.zero_grad()
    loss.backward()
    solver.step()

    # Print and plot every now and then
    if it % 1000 == 0:
        print('Iter-{}; Loss: {:.4}'.format(it, loss.item()))

        z = Variable(torch.randn(mb_size, z_dim))
        samples = P(z).data.numpy()[:16]

        fig = plt.figure(figsize=(4, 4))
        gs = gridspec.GridSpec(4, 4)
        gs.update(wspace=0.05, hspace=0.05)

        for i, sample in enumerate(samples):
            ax = plt.subplot(gs[i])
            plt.axis('off')
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

        if not os.path.exists('out/'):
            os.makedirs('out/')

        plt.savefig('out/{}.png'.format(str(c).zfill(3)), bbox_inches='tight')
        c += 1
        plt.close(fig)

100%|██████████| 9912422/9912422 [00:00<00:00, 73145678.79it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 7538002.10it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 32333869.73it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9963665.67it/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw




/usr/local/lib/python3.10/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Iter-0; Loss: 769.7
Iter-1000; Loss: 153.5
Iter-2000; Loss: 135.9
Iter-3000; Loss: 123.4
Iter-4000; Loss: 134.7
Iter-5000; Loss: 135.8
Iter-6000; Loss: 141.2
Iter-7000; Loss: 133.8
Iter-8000; Loss: 150.0
Iter-9000; Loss: 133.0
Iter-10000; Loss: 124.1
Iter-11000; Loss: 132.4
Iter-12000; Loss: 135.5
Iter-13000; Loss: 136.3
Iter-14000; Loss: 133.4
Iter-15000; Loss: 134.0
Iter-16000; Loss: 139.9
Iter-17000; Loss: 134.5
Iter-18000; Loss: 136.4
Iter-19000; Loss: 116.4
Iter-20000; Loss: 123.2
Iter-21000; Loss: 129.3
Iter-22000; Loss: 137.4
Iter-23000; Loss: 126.1
Iter-24000; Loss: 127.1
Iter-25000; Loss: 126.6
Iter-26000; Loss: 131.3
Iter-27000; Loss: 122.3
Iter-28000; Loss: 126.7
Iter-29000; Loss: 128.5
Iter-30000; Loss: 133.5
Iter-31000; Loss: 137.9
Iter-32000; Loss: 134.8
Iter-33000; Loss: 140.6
Iter-34000; Loss: 136.3
Iter-35000; Loss: 124.8
Iter-36000; Loss: 133.2
Iter-37000; Loss: 134.5
Iter-38000; Loss: 128.9
Iter-39000; Loss: 126.6
Iter-40000; Loss: 133.7
Iter-41000; Loss: 125.7
Iter-